In [1]:
import numpy as np
import csv

In [2]:
#Abrir el archivo de datos

with open('crime_data.csv', 'r') as file:
    reader = csv.reader(file)
    
    rows = []
    data = []
    
    for row in reader:
        rows.append(row)
        
    for i in range (51057):
        data.append(np.array(rows[i][3:], dtype = float))

ValueError: could not convert string to float: 'lat'

In [3]:
#Funciones del proyecto a utilizar 
def grad(f, x0):
    
    n = x0.size
    eps = 0.00001
    res = np.zeros(n)
    
    for i in range(n):
        aux = np.zeros(n)
        aux[i] = eps
        x1 = x0 + aux
        res[i] = (f(x1) - f(x0)) / eps   
        
    return res

In [4]:
def hess(f,x0):
    
    n = x0.size
    eps = 0.00001
    res = np.zeros((n,n))
    
    for i in range(n):
        
        aux = np.zeros(n)
        aux[i] = eps
        x1 = x0 + aux
        res[:,i] = (grad(f,x1) - grad(f,x0)) / eps
        
    return res

In [5]:
def alfa(f,x0,p):
    
    ro = 0.8
    c = 0.0001
    alpha = 1
    
    while f(x0 + alpha * p) > f(x0) + c * alpha * (grad(f,x0).T).dot(p):
        
        alpha = ro * alpha
        
    return alpha

In [6]:
def calcula_gamma(A):
    
    k = 100
    n = A.shape[0]
    beta = 0.001
    
    diagonal = np.diagonal(A)
    minimo = np.amin(diagonal)
    
    if minimo > 0:
        gamma = 0
    else:
        gamma = -minimo + beta
    #end if
    
    for i in range(k):    
        
        try:
            np.linalg.cholesky(A + gamma * np.identity(n))
        except np.linalg.LinAlgError:
            gamma = max([2 * gamma, beta])
        else:
            break
            
    return gamma

In [7]:
def newton_mod(f,x0):
    
    k = 1000
    n = x0.size
    
    for i in range(k):
        
        gamma = calcula_gamma(hess(f, x0))
        B0 = hess(f, x0) + gamma * np.identity(n)
        
        p0 = -np.linalg.inv(B0).dot(grad(f,x0))        
        alpha = alfa(f, x0, p0)
        
        x0 = x0 + alpha * p0
        
        #para ver si la función realmente decrece
        #print(f(x0)) 
        
    return x0

In [8]:
def ubicacion_camaras(data):
    
    n = 8000 #Número de cámaras
    m = 31057 #Número de delitos
    c = np.zeros((n,2)) #Matriz de las ubicaciones, la primera entrada es para la latitud y la segunda para la longitud
    
    for i in range (n):
        for j in range (m):
            c1 = np.array([0,0]) #Ubicación inicial 
            c[i,:] = newton_mod(np.linalg,norm(d[j],c),c1)
    
    return c